In [1]:
import sys
sys.path.insert(0, '../' )


In [2]:


from Crypto.Cipher import AES

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct = AES.new( sk , AES.MODE_ECB ).encrypt( pt )
#ct, rd_states , rk , rk_tmp = aes128.aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( "ct0", ct )



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
ct0 b'i\xc4\xe0\xd8j{\x040\xd8\xcd\xb7\x80p\xb4\xc5Z'


In [3]:
from aesR1CS import aes128R1CS_z as R1CSz

z = R1CSz.get_vec_z(pt,sk,64)

print( R1CSz.aes128R1CS_num_constrains , len(z) )


3656 3320


In [4]:
from aesR1CS import r1cs
import numpy as np

MATs = r1cs.get_r1cs( R1CSz.aes128R1CS_num_constrains , len(z) , 64, 128)  # AES128

_mat_a = MATs['A']
_mat_b = MATs['B']
_mat_c = MATs['C']

mat_a = np.array( _mat_a )
mat_b = np.array( _mat_b )
mat_c = np.array( _mat_c )



In [5]:

from m_aurora import *

##### test ##########

proof = generate_proof( (mat_a,mat_b,mat_c,z,64) , b'123' )


m(#rows): 3656 x n: 3320, witness_idx: 64, pad_len: 4096
pad_len: 4096, inst_dim: 6
padding and calcuate f_1v, f_z, f_w
time: 0.14434504508972168 secs
f_z : [4096] ...[60:68]: [11876712511685630227, 1490032541726768366, 332458469872254918, 200, 18174738647891815068, 6001071716487184560, 13518469962322655972, 233317187178767899] ...
f_w : (f_z - f_1v) /Z_6 :  [4071544815287063751429570163657677555134831481824947510829, 1263211062984712749411083712164085838059225111420963806995, 2767837989273261907368810399861902751714150938860792364978, 404027588473964302064485763162088008663268290056040345038] ...
calculate Az, Bz, Cz
time: 4.423435688018799 secs
calculate f_Az, f_Bz, f_Cz
time: 0.02473902702331543 secs
rowcheck
r_lincheck: [8192]: ...[4094:4098] ... [5720443651321118019961721256494769897591057341711528765633, 0, 6119622517117424670597151491431251791674330345767766709066, 3085676348056205675044418416084586383148781459222013443379]
commit f_w, f_Az, f_Bz , f_Cz, r_lincheck , r_ldt
commi

In [6]:


verify_proof( proof , (mat_a,mat_b,mat_c,z[:64],64) , b'123' )


m(#rows): 3656 x n: 3320, witness_idx: 64, pad_len: 4096
recover challenges
check if commits are opened correctly
all passed
verify ldt
True
prepare f_alpha, p2A, p2B, p2C
lin-check starts. calculate f_alpha, v_alpha
time: 0.02183818817138672 secs
lin-check step 1. calculate p2A, p2B, p2C and evaluate their values
evaluate values of p2A, p2B, p2C
time: 13.55114483833313 secs
interpolate p2A, p2B, p2C
time: 0.012048959732055664 secs
return f_alpha, p2A , p2B , p2C
RS codeword of polynomials
check virtual oracle consistency
all passed


True